In [73]:
import numpy as np
import pandas as pd
import akshare as ak
from scipy.optimize import minimize

etfs = {
#         'sh511360': '短融ETF',
#         'sh511520': '政金债券ETF',
#         'sh511380': '可转债ETF',    
#         'sz159651': '创业板ETF',
        'sh513500': '标普500ETF',
#         'sh511260': '十年国债ETF',
        'sh511010': '国债ETF',    
        'sh518880': '黄金ETF',
#         'sh511060': '5年地方债ETF',
#         'sh512800': '银行ETF',    
#         'sh512880': '证券ETF',
        'sh510300': '沪深300ETF',
#         'sh510500': '中证500ETF',    
#         'sz159949': '创业板50ETF',
#         'sh588080': '科创板50ETF',
#         'sh512100': '中证1000ETF',    
        'sz162411': '华宝油气ETF'}# 获取ETF历史净值数据

data = pd.DataFrame()
for code in etfs.keys():    
    fund_data = ak.fund_etf_hist_sina(symbol=code)    
    fund_data.set_index('date',inplace=True)    
    data[code] = fund_data.close

data = data.dropna()

In [84]:
cov_matrix 

,sh513500,sh511010,sh518880,sh510300,sz162411
sh513500,3.044493e-04,-9.039754e-07,-6.331998e-07,3.524906e-05,0.000090
sh511010,-9.039754e-07,2.613854e-06,1.775179e-06,-2.832673e-06,-0.000002
sh518880,-6.331998e-07,1.775179e-06,6.626061e-05,3.170440e-07,0.000008
sh510300,3.524906e-05,-2.832673e-06,3.170440e-07,2.099271e-04,0.000067
sz162411,8.966962e-05,-2.355836e-06,8.068820e-06,6.702761e-05,0.000459


In [87]:
returns = np.log(data / data.shift(1))
mean_returns = returns.mean()
cov_matrix = returns.cov()
corr_matrix = returns.corr()
num_assets = len(data.columns)
init_guess = np.repeat(1/num_assets,num_assets)
def portfolio_volatility(weights):    
    return np.sqrt(np.dot(weights.T,np.dot(cov_matrix,weights)))

def portfolio_correlation(weights):    
    return np.sum(np.dot(np.diag(weights),np.dot(corr_matrix,np.diag(weights))))

def risk_contribution(weights):    
    mc_risk = portfolio_volatility(weights)    
    mc_risk_contrib = (weights * (cov_matrix @ weights)) / mc_risk    
    risk_target = mc_risk / num_assets    
    risk_diffs = mc_risk_contrib - risk_target    
    return np.sum(np.square(risk_diffs))

def risk_budget_objective(weights,cov_matrix):
    weights=np.array(weights)#weights为一维数组
    sigma=np.sqrt(np.dot(weights,np.dot(cov_matrix,weights))) #获取组合标准差   
    #sigma = np.sqrt(weights@cov@weights)
    MRC=np.dot(cov_matrix,weights)/sigma #MRC = cov@weights/sigma
    #MRC = np.dot(weights,cov)/sigma
    TRC=weights*MRC
    delta_TRC=[sum((i-TRC)**2) for i in TRC]
    return sum(delta_TRC)

def total_weight_constraint(weights):    
    return np.sum(weights) - 1

def long_only_constraint(weights):    
    return weights

constraints = ({'type':'eq','fun':total_weight_constraint},               
               {'type':'ineq','fun':long_only_constraint})

def objective(weights):   
    return portfolio_volatility(weights) + 0.2 * portfolio_correlation(weights)

optimal_result = minimize(risk_budget_objective,                          
                          init_guess,
                          args=(cov_matrix),
                          method='SLSQP',                          
                          constraints=constraints,                          
                          options={'disp':False})

weights = optimal_result.x
weights = pd.Series(weights,index=data.columns)

In [88]:
port_returns = returns.dot(weights)
cumret = (port_returns + 1).cumprod()
annual_return = port_returns.mean() * 252
annual_volatility = port_returns.std() * np.sqrt(252)
sharpe_ratio = annual_return / annual_volatility
max_drawdown = (cumret.cummax() - cumret).max()
calmar_ratio = annual_return / max_drawdown

print(f"全天候组合资产权重:\n{weights}\n")
print(f"年化收益率:{annual_return:.2%}")
print(f"年化波动率:{annual_volatility:.2%}")  
print(f"夏普比率:{sharpe_ratio:.2f}")
print(f"最大回撤:{max_drawdown:.2%}")
print(f"卡玛比率:{calmar_ratio:.2f}")

全天候组合资产权重:
sh513500    0.2
sh511010    0.2
sh518880    0.2
sh510300    0.2
sz162411    0.2
dtype: float64

年化收益率:3.78%
年化波动率:12.02%
夏普比率:0.31
最大回撤:25.40%
卡玛比率:0.15


In [29]:
import pandas as pd
import numpy as np
import akshare as ak
from scipy.optimize import minimize

# 读入5支股票 2015-01-01 到 2021-12-31 日收盘价数据，并计算对数收益率
def get_ret(code):
    data=ak.stock_zh_a_hist(symbol=code,period="daily",start_date="20180101",end_date='20211231',adjust="")
    data.index=pd.to_datetime(data['日期'],format='%Y-%m-%d')
    close=data['收盘']#日收盘价
    close.name=code
    ret=np.log(close/close.shift(1))#日收益率
    return ret

codes=['000001','000651','300015','600519','000625']
ret=pd.DataFrame()
for code in codes:
    ret_= get_ret(code)
    ret=pd.concat([ret,ret_],axis=1)

ret=ret.dropna()

R_cov=ret.cov()#计算协方差
cov=np.array(R_cov)

In [45]:
data=ak.stock_zh_a_hist(symbol='000001',period="daily",start_date="20150101",end_date='20211231',adjust="")
data

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2015-01-05,000001,15.99,16.02,16.28,15.60,2860436,4.565388e+09,4.29,1.14,0.18,2.91
1,2015-01-06,000001,15.85,15.78,16.39,15.55,2166421,3.453446e+09,5.24,-1.50,-0.24,2.20
2,2015-01-07,000001,15.56,15.48,15.83,15.30,1700121,2.634796e+09,3.36,-1.90,-0.30,1.73
3,2015-01-08,000001,15.50,14.96,15.57,14.90,1407714,2.128003e+09,4.33,-3.36,-0.52,1.43
4,2015-01-09,000001,14.90,15.08,15.87,14.71,2508500,3.835378e+09,7.75,0.80,0.12,2.55
...,...,...,...,...,...,...,...,...,...,...,...,...
1700,2021-12-27,000001,17.33,17.22,17.35,17.16,731119,1.260455e+09,1.10,-0.52,-0.09,0.38
1701,2021-12-28,000001,17.22,17.17,17.33,17.09,1126639,1.934461e+09,1.39,-0.29,-0.05,0.58
1702,2021-12-29,000001,17.16,16.75,17.16,16.70,1469374,2.480535e+09,2.68,-2.45,-0.42,0.76
1703,2021-12-30,000001,16.76,16.82,16.95,16.72,796664,1.342374e+09,1.37,0.42,0.07,0.41


In [36]:
def risk_budget_objective(weights,cov):
    weights=np.array(weights)#weights为一维数组
    sigma=np.sqrt(np.dot(weights,np.dot(cov,weights))) #获取组合标准差   
    #sigma = np.sqrt(weights@cov@weights)
    MRC=np.dot(cov,weights)/sigma #MRC = cov@weights/sigma
    #MRC = np.dot(weights,cov)/sigma
    TRC=weights*MRC
    delta_TRC=[sum((i-TRC)**2) for i in TRC]
    return sum(delta_TRC)

def total_weight_constraint(x):
    return np.sum(x)-1.0

x0 = np.ones(cov.shape[0])/cov.shape[0]
bnds=tuple((0,None) for x in x0)
cons=({'type':'eq','fun':total_weight_constraint})
#cons=({'type':'eq','fun':lambdax:sum(x)-1})
options={'disp':False,'maxiter':1000,'ftol':1e-20}

solution=minimize(risk_budget_objective,x0,args=(cov),bounds=bnds,constraints=cons,method='SLSQP',options=options)

# 求解出权重
final_weights=solution.x #权重
for i in range(len(final_weights)):
    print(f'{final_weights[i]:.1%}投资于{R_cov.columns[i]}')

24.3%投资于000001
16.7%投资于000651
16.1%投资于300015
24.8%投资于600519
18.1%投资于000625


In [67]:
cov

array([[0.00046516, 0.00025545, 0.00019175, 0.00019078, 0.00021711],
       [0.00025545, 0.00102789, 0.00028302, 0.00025856, 0.00029998],
       [0.00019175, 0.00028302, 0.00125436, 0.00026395, 0.00029353],
       [0.00019078, 0.00025856, 0.00026395, 0.00042608, 0.00016959],
       [0.00021711, 0.00029998, 0.00029353, 0.00016959, 0.00096571]])

## risk parity and annual rebalance

In [49]:
data

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2015-01-05,000001,15.99,16.02,16.28,15.60,2860436,4.565388e+09,4.29,1.14,0.18,2.91
1,2015-01-06,000001,15.85,15.78,16.39,15.55,2166421,3.453446e+09,5.24,-1.50,-0.24,2.20
2,2015-01-07,000001,15.56,15.48,15.83,15.30,1700121,2.634796e+09,3.36,-1.90,-0.30,1.73
3,2015-01-08,000001,15.50,14.96,15.57,14.90,1407714,2.128003e+09,4.33,-3.36,-0.52,1.43
4,2015-01-09,000001,14.90,15.08,15.87,14.71,2508500,3.835378e+09,7.75,0.80,0.12,2.55
...,...,...,...,...,...,...,...,...,...,...,...,...
1700,2021-12-27,000001,17.33,17.22,17.35,17.16,731119,1.260455e+09,1.10,-0.52,-0.09,0.38
1701,2021-12-28,000001,17.22,17.17,17.33,17.09,1126639,1.934461e+09,1.39,-0.29,-0.05,0.58
1702,2021-12-29,000001,17.16,16.75,17.16,16.70,1469374,2.480535e+09,2.68,-2.45,-0.42,0.76
1703,2021-12-30,000001,16.76,16.82,16.95,16.72,796664,1.342374e+09,1.37,0.42,0.07,0.41


In [66]:
import backtrader as bt
import pandas as pd
import numpy as np
import akshare as ak
from scipy.optimize import minimize

# 读入股票收盘价数据，并计算对数收益率
def get_daily_quote(code):
    data = ak.stock_zh_a_hist(symbol=code, period="daily", start_date="20150101", end_date='20211231', adjust="")
    data.index = pd.to_datetime(data['日期'], format='%Y-%m-%d')
    df_daily_quote = data[['开盘','收盘','最高','最低','成交量','涨跌幅']]  # 日收盘价
    df_daily_quote.columns=['open','close','high','low','volume','pct_change']
    return df_daily_quote

# 风险预算目标函数
def risk_budget_objective(weights, cov):
    weights = np.array(weights)
    sigma = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    MRC = np.dot(cov, weights) / sigma
    TRC = weights * MRC
    delta_TRC = [i - TRC for i in TRC]
    return sum([i**2 for i in delta_TRC])

# 权重总和约束
def total_weight_constraint(weights):
    return np.sum(weights) - 1.0

# 策略类
class RiskParityStrategy(bt.Strategy):
    params = (('annual_rebalance', True),)

    def __init__(self):
        self.dataclose = [d.close for d in self.datas]
        self.last_rebalance = None
        self.portfolio_values = {}
        self.factor_exposures = {}
        self.factor_returns = {}
        self.positions_tracker = {}
        
    def log(self, txt, dt=None):
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float):
            dt = bt.num2date(dt)
        print("%s, %s" % (dt.date(), txt))

    def next(self):
        current_date = self.data.datetime.date()
        last_year_date = current_date - pd.Timedelta(days=365)
        total_value = self.broker.getvalue()
        self.portfolio_values[current_date] = total_value
        self.calculate_factor_exposures()

        # Store positions for each asset
        positions_snapshot = {}
        for d in self.datas:
            position = self.getposition(d)
            current_value = position.size * d.close[0]
            positions_snapshot[d._name] = current_value

        positions_snapshot['cash']= self.broker.get_cash()
        self.positions_tracker[current_date] = positions_snapshot

        if self.last_rebalance is None or current_date.year > self.last_rebalance.year:
            self.log(f"Rebalancing on {current_date}")
            self.rebalance_portfolio()
            self.last_rebalance = current_date
            
    def calculate_factor_exposures(self):
        total_value = self.broker.getvalue()
        exposures = {}
        for d in self.datas:
            current_value = d.close[0] * self.broker.getposition(d).size
            exposures[d._name] = current_value / total_value
        self.factor_exposures[self.data.datetime.date()] = exposures

    def rebalance_portfolio(self):
        # 获取过去一年的数据
        current_date = self.data.datetime.date()
        last_year_date = current_date - pd.Timedelta(days=365)
        ret_dataframe={}
        for d in self.datas:
            ticker = d._name
            data_series = pd.Series(d.pct_change.get(size=len(d)),
                                    index=[bt.num2date(d.datetime[i]) for i in range(len(d))],
                                    name='pct_change')
            ret_dataframe[ticker] = data_series.loc[last_year_date:current_date]
        
        ret = pd.DataFrame(ret_dataframe)/100
        ret = ret.dropna()
        R_cov = ret.cov()  # 计算协方差
        cov = np.array(R_cov)

        # 优化权重
        x0 = np.ones(cov.shape[0]) / cov.shape[0]
        bnds = tuple((0, None) for x in x0)
        cons = ({'type': 'eq', 'fun': total_weight_constraint})
        options = {'disp': False, 'maxiter': 1000, 'ftol': 1e-20}
        solution = minimize(risk_budget_objective, x0, args=(cov), bounds=bnds, constraints=cons, method='SLSQP', options=options)
        final_weights = solution.x

        # 调整投资组合权重
        for i, stock in enumerate(self.datas):
            self.order_target_percent(target=final_weights[i], data=stock)

            
class CustomFundData(bt.feeds.PandasData):
    # Define the columns you want to use, and set the corresponding line to None if you don't want to use it
    lines = ('pct_change',)

    params = (
        ('datetime', None),  # This field is not used because the index is the date
        ('pct_change', -1),          # Map the pct change
    )
if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # 添加数据
    codes = ['000001', '000651', '300015', '600519', '000625']
    for code in codes:
        ret_ = get_daily_quote(code)
        data = CustomFundData(dataname=ret_)
        cerebro.adddata(data, name=code)

    # 添加策略
    cerebro.addstrategy(RiskParityStrategy)

    # 设置初始资金
    cerebro.broker.setcash(100000.0)

    # 设置佣金为0.1%
    cerebro.broker.setcommission(commission=0.001)

    # 运行回测
    cerebro.run()

    # 绘制结果
    cerebro.plot()

2015-01-05, Rebalancing on 2015-01-05


D:\Anaconda\lib\site-packages\pandas\core\frame.py:9679: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: Objective function must return a scalar

In [61]:
ret_

,open,close,high,low,volume,pct_change
日期,,,,,,
2015-01-05,16.40,18.07,18.07,16.32,820880,9.98
2015-01-06,18.10,19.36,19.88,18.10,1200591,7.14
2015-01-07,18.50,19.50,19.84,18.40,556700,0.72
2015-01-08,19.46,19.70,20.44,19.14,445859,1.03
2015-01-09,19.65,18.85,20.15,18.70,517412,-4.31
...,...,...,...,...,...,...
2021-12-27,15.30,15.12,15.46,14.80,854119,-1.75
2021-12-28,15.20,15.49,15.52,15.08,684913,2.45
2021-12-29,15.50,15.20,15.54,15.15,575000,-1.87
